# Hosting Strands Agents with Amazon Bedrock models in Amazon Bedrock AgentCore Runtime

## Overview

In this tutorial we will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime. We will provide examples using Amazon Bedrock models and non-Bedrock models such as Azure OpenAI and Gemini.


### Tutorial Details

### Tutorial Architecture

In this tutorial we will describe how to deploy an existing agent to AgentCore runtime. 

For demonstration purposes, we will  use a Strands Agent using Amazon Bedrock models

In our example we will use a very simple agent with two tools: `get_weather` and `get_time`. 

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### Tutorial Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime
* Using Amazon Bedrock models
* Using Strands Agents


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents

In [ ]:
!pip install -r requirements.txt

## Creating your agents and experimenting locally

Before we deploy our agents to AgentCore Runtime, let's develop and run them locally for experimentation purposes.

For production agentic applications we will need to decouple the agent creation process from the agent invocation one. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. Let's first look how each agent is developed during the experimentation phase.

The architecture here will look as following:

<div style="text-align:left">
    <img src="images/architecture_local.png" width="50%"/>
</div>

In [ ]:
%%writefile strands_agent.py
from strands import Agent, tool
import argparse
import json
from strands.models import BedrockModel
from dotenv import load_dotenv
import os
from dotenv import load_dotenv
import boto3
from datetime import datetime

# Load environment variables
load_dotenv()



# Check AWS credentials and boto sessions 


# PTO data
PTO_HOURS_AVAILABLE = 94

try:
    # Create a Bedrock client to test credentials
    bedrock_client = boto3.client('bedrock-runtime', region_name='us-west-1')
    
    # Try to get caller identity to verify credentials
    sts = boto3.client('sts', region_name='us-west-2')
    identity = sts.get_caller_identity()
    

    
    # If using temporary credentials, check expiration
    session = boto3.Session()
    credentials = session.get_credentials()

    
except Exception as e:
    print(f"❌ AWS Credentials Error: {e}")
    print("\n🔧 To fix this:")
    print("   1. Run: aws configure")
    print("   2. Or update your .env file with new AWS credentials")
    print("   3. Or get fresh credentials from AWS SSO/Academy")
    print("\n   Then restart the kernel and run from the beginning.")


# Define tools using if-else logic with the @tool decorator
@tool
def get_available_pto_hours() -> str:
    """Get the total number of PTO hours available."""
    return f"You have {PTO_HOURS_AVAILABLE} PTO hours available."

@tool
def get_available_pto_days() -> str:
    """Convert PTO hours to days (assuming 8-hour workday)."""
    days = PTO_HOURS_AVAILABLE / 8
    return f"You have {days} PTO days available ({PTO_HOURS_AVAILABLE} hours)."

@tool
def can_take_full_day() -> str:
    """Check if employee has enough PTO for a full day off (8 hours)."""
    if PTO_HOURS_AVAILABLE >= 8:
        return f"Yes, you can take a full day off. You have {PTO_HOURS_AVAILABLE} hours available."
    else:
        return f"No, you don't have enough PTO for a full day. You only have {PTO_HOURS_AVAILABLE} hours available."

@tool
def can_take_half_day() -> str:
    """Check if employee has enough PTO for a half day off (4 hours)."""
    if PTO_HOURS_AVAILABLE >= 4:
        return f"Yes, you can take a half day off. You have {PTO_HOURS_AVAILABLE} hours available."
    else:
        return f"No, you don't have enough PTO for a half day. You only have {PTO_HOURS_AVAILABLE} hours available."


try:
    model = BedrockModel(
        model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
        boto_session=session
    )

except Exception as e:
    print(f"⚠️  Error initializing Bedrock model: {e}")
    print("\n🔧 Troubleshooting:")
    print("1. Check AWS credentials are valid (run cell 5)")
    print("2. Enable Claude Sonnet 4 in Bedrock console → Model access")
    print("3. Use inference profile (us.anthropic.claude-sonnet-4-v1:0)")
    print("4. Try different region if needed (us-east-1, us-west-2)")


# Define the system prompt for the PTO agent
system_prompt = """You are a helpful HR assistant that helps employees check their PTO (Paid Time Off) hours. 
Be friendly, concise, and professional. Use the available tools to provide accurate information about PTO balances."""

# Create the PTO agent with tools
pto_agent = Agent(
    model=model,
    system_prompt=system_prompt,
    tools=[
        get_available_pto_hours,
        get_available_pto_days,
        can_take_full_day,
        can_take_half_day
    ]
)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    payload = json.loads(args.payload)
    response = pto_agent(payload.get("prompt"))
    print(response.message['content'][0]['text'])

#### Invoking local agent

In [ ]:
!python strands_agent.py '{"prompt": "How many days of pto do I have?"}'

## Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agents to AgentCore Runtime. To do so we need to:
* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint` decorator
* Let AgentCoreRuntime control the running of the agent with `app.run()`

### Strands Agents with Amazon Bedrock model
Let's start with our Strands Agent using Amazon Bedrock model. All the others will work exactly the same.

In [ ]:
%%writefile strands_agentcore_bedrock.py
from strands import Agent, tool
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel

# app = BedrockAgentCoreApp()

# PTO data
PTO_HOURS_AVAILABLE = 94


# Define tools using @tools decorator
@tool
def get_available_pto_hours() -> str:
    """Get the total number of PTO hours available."""
    return f"You have {PTO_HOURS_AVAILABLE} PTO hours available."

@tool
def get_available_pto_days() -> str:
    """Convert PTO hours to days (assuming 8-hour workday)."""
    days = PTO_HOURS_AVAILABLE / 8
    return f"You have {days} PTO days available ({PTO_HOURS_AVAILABLE} hours)."

@tool
def can_take_full_day() -> str:
    """Check if employee has enough PTO for a full day off (8 hours)."""
    if PTO_HOURS_AVAILABLE >= 8:
        return f"Yes, you can take a full day off. You have {PTO_HOURS_AVAILABLE} hours available."
    else:
        return f"No, you don't have enough PTO for a full day. You only have {PTO_HOURS_AVAILABLE} hours available."

@tool
def can_take_half_day() -> str:
    """Check if employee has enough PTO for a half day off (4 hours)."""
    if PTO_HOURS_AVAILABLE >= 4:
        return f"Yes, you can take a half day off. You have {PTO_HOURS_AVAILABLE} hours available."
    else:
        return f"No, you don't have enough PTO for a half day. You only have {PTO_HOURS_AVAILABLE} hours available."


# Initialize model
model = BedrockModel(model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0")


# Create agent
agent = Agent(
    model=model,
    system_prompt="You are a helpful HR assistant that helps employees check their PTO (Paid Time Off) hours. Be friendly, concise, and professional.",
    tools=[get_available_pto_hours, get_available_pto_days, can_take_full_day, can_take_half_day]
)

# @app.entrypoint
def strands_bedrock_agentcore(payload):
    """Invoke the agent with a payload"""
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCore Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Configure AgentCore Runtime deployment

First we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [2]:
from bedrock_agentcore_starter_toolkit import Runtime
import boto3
from boto3.session import Session
from dotenv import load_dotenv

load_dotenv()
sts = boto3.client('sts', region_name='us-west-2')
identity = sts.get_caller_identity()
    
print("✓ AWS Credentials are valid!")
print(f"  Account: {identity['Account']}")
print(f"  User ARN: {identity['Arn']}")
    
    # If using temporary credentials, check expiration
session = boto3.Session()
credentials = session.get_credentials()
region = session.region_name


agentcore_runtime = Runtime()
agent_name = "strands_claude_getting_started"
response = agentcore_runtime.configure(
    entrypoint="strands_agentcore_bedrock.py",  
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

Entrypoint parsed: file=/Users/chuongho/Library/CloudStorage/WorkDocsDrive-Documents/Workdocks - Working Files/Lunch and Learn/lunch_and_learn_demos/10-strands-agent-on-agentcore/strands_agentcore_bedrock.py, bedrock_agentcore_name=strands_agentcore_bedrock
Configuring BedrockAgentCore agent: strands_claude_getting_started


✓ AWS Credentials are valid!
  Account: 977099023120
  User ARN: arn:aws:sts::977099023120:assumed-role/AWSReservedSSO_AdministratorAccess_8133186339cd89a9/chuongho@myhofamily.net


⚠️  ℹ️  No container engine found (Docker/Finch/Podman not installed)
✅ Default deployment uses CodeBuild (no container engine needed)
💡 Run 'agentcore launch' for cloud-based building and deployment
💡 For local builds, install Docker, Finch, or Podman

Generated Dockerfile: /Users/chuongho/Library/CloudStorage/WorkDocsDrive-Documents/Workdocks - Working Files/Lunch and Learn/lunch_and_learn_demos/10-strands-agent-on-agentcore/Dockerfile
Generated .dockerignore: /Users/chuongho/Library/CloudStorage/WorkDocsDrive-Documents/Workdocks - Working Files/Lunch and Learn/lunch_and_learn_demos/10-strands-agent-on-agentcore/.dockerignore
Keeping 'strands_claude_getting_started' as default agent
Bedrock AgentCore configured: /Users/chuongho/Library/CloudStorage/WorkDocsDrive-Documents/Workdocks - Working Files/Lunch and Learn/lunch_and_learn_demos/10-strands-agent-on-agentcore/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/Users/chuongho/Library/CloudStorage/WorkDocsDrive-Documents/Workdocks - Working Files/Lunch and Learn/lunch_and_learn_demos/10-strands-agent-on-agentcore/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/Users/chuongho/Library/CloudStorage/WorkDocsDrive-Documents/Workdocks - Working Files/Lunch and Learn/lunch_and_learn_demos/10-strands-agent-on-agentcore/Dockerfile'), dockerignore_path=PosixPath('/Users/chuongho/Library/CloudStorage/WorkDocsDrive-Documents/Workdocks - Working Files/Lunch and Learn/lunch_and_learn_demos/10-strands-agent-on-agentcore/.dockerignore'), runtime='None', region='us-west-2', account_id='977099023120', execution_role=None, ecr_repository=None, auto_create_ecr=True)

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [3]:
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'strands_claude_getting_started' to account 977099023120 (us-west-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: strands_claude_getting_started
✅ ECR repository available: 977099023120.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_claude_getting_started
Getting or creating execution role for agent: strands_claude_getting_started
Using AWS region: us-west-2, account ID: 977099023120
Role name: AmazonBedrockAgentCoreSDKRuntime-us-west-2-8556fc4504


✅ Reusing existing ECR repository: 977099023120.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_claude_getting_started


✅ Reusing existing execution role: arn:aws:iam::977099023120:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-8556fc4504
✅ Execution role available: arn:aws:iam::977099023120:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-8556fc4504
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: strands_claude_getting_started
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-8556fc4504
Reusing existing CodeBuild execution role: arn:aws:iam::977099023120:role/AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-8556fc4504
Using .dockerignore with 44 patterns
Uploaded source to S3: strands_claude_getting_started/source.zip
Updated CodeBuild project: bedrock-agentcore-strands_claude_getting_started-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.1s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 9.7s
🔄 DOWNLOAD_SOURCE started (t

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [4]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: strands_claude_getting_started


'READY'

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [5]:
invoke_response = agentcore_runtime.invoke({"prompt": "how many pto hours do I have?"})
invoke_response

{'ResponseMetadata': {'RequestId': '80c20922-7778-4a66-839d-28c8ddeaab46',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 31 Oct 2025 16:41:40 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': '80c20922-7778-4a66-839d-28c8ddeaab46',
   'baggage': 'Self=1-6904e6be-39c8640d06ca65ac3becf9de,session.id=ccd179e3-53c8-4a5f-ad77-ae26096bfeb4',
   'x-amzn-bedrock-agentcore-runtime-session-id': 'ccd179e3-53c8-4a5f-ad77-ae26096bfeb4',
   'x-amzn-trace-id': 'Root=1-6904e6be-58e7414b5038646672ebea8a;Parent=7d6d91e02756abb9;Sampled=1;Self=1-6904e6be-39c8640d06ca65ac3becf9de'},
  'RetryAttempts': 0},
 'runtimeSessionId': 'ccd179e3-53c8-4a5f-ad77-ae26096bfeb4',
 'traceId': 'Root=1-6904e6be-58e7414b5038646672ebea8a;Parent=7d6d91e02756abb9;Sampled=1;Self=1-6904e6be-39c8640d06ca65ac3becf9de',
 'baggage': 'Self=1-6904e6be-39c8640d06ca65ac3becf9de,session.id=ccd179e3-53c8-4a5f-ad77-ae26096bfeb4',
 'contentType': 

### Let's process the response results because the above is just ugly

In [6]:
from IPython.display import Markdown, display
import json
response_text = invoke_response['response'][0]
print(response_text)


11.75 days, assuming an 8-hour workday. Is there anything else you'd like to know about your PTO?


In [ ]:
#get boto3 session data
import os
import boto3
from datetime import datetime
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

try:
    # Create a Bedrock client to test credentials
    bedrock_client = boto3.client('bedrock-runtime', region_name='us-west-2')
    
    # Try to get caller identity to verify credentials
    sts = boto3.client('sts', region_name='us-west-1')
    identity = sts.get_caller_identity()
    
    print("✓ AWS Credentials are valid!")
    print(f"  Account: {identity['Account']}")
    # print(f"  region: {identity['Region_Name']}")
    
    # If using temporary credentials, check expiration
    session = boto3.Session()
    credentials = session.get_credentials()
    print("\n⚠️  Session has been created")

except Exception as e:
    print(f"❌ AWS Credentials Error: {e}")
    print("\n🔧 To fix this:")
    print("   1. Run: aws configure")
    print("   2. Or update your .env file with new AWS credentials")
    print("   3. Or get fresh credentials from AWS SSO/Academy")
    print("\n   Then restart the kernel and run from the beginning.")

### Invoking AgentCore Runtime with boto3

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it.

In [7]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "How many pto days do I have?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

You currently have 11.75 PTO days available to use. This is equivalent to 94 hours of paid time off.

Is there anything else you'd like to know about your PTO balance?

## Let's try to host the agent behind a front end

In [ ]:
import gradio as gr
import boto3
import json
from bedrock_agentcore_starter_toolkit import Runtime

# Initialize AgentCore Runtime
#agentcore_runtime = Runtime()

agentcore_runtime = Runtime()
agent_name = "strands_claude_getting_started"
response = agentcore_runtime.configure(
    entrypoint="strands_agentcore_bedrock.py",  
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response


# Check if agent is deployed
try:
    status_response = agentcore_runtime.status()
    # Get agent_arn from the status response
    if hasattr(status_response, 'agent_arn') and status_response.agent_arn:
        agent_arn = status_response.agent_arn
    else:
        # Fallback: try to get from config file
        import yaml
        with open('.bedrock_agentcore.yaml', 'r') as f:
            config = yaml.safe_load(f)
            agent_arn = config['agents']['strands_claude_getting_started']['bedrock_agentcore']['agent_arn']
    
    region = boto3.Session().region_name or 'us-west-2'
    agentcore_client = boto3.client('bedrock-agentcore', region_name=region)
    print(f"✓ Connected to AgentCore Runtime: {agent_arn}")
except Exception as e:
    print(f"❌ Error connecting to AgentCore Runtime: {e}")
    agent_arn = None
    agentcore_client = None

def query_agentcore_runtime(user_query):
    """
    Query the deployed AgentCore Runtime using boto3
    """
    if not agent_arn or not agentcore_client:
        return "❌ AgentCore Runtime not available.\n\nPlease complete these steps:\n1. Run agentcore_runtime.configure()\n2. Run agentcore_runtime.launch()\n3. Wait for status to be 'READY'\n\nThen restart this interface."
    
    try:
        # Call the AgentCore Runtime using boto3
        response = agentcore_client.invoke_agent_runtime(
            agentRuntimeArn=agent_arn,
            qualifier="DEFAULT",
            payload=json.dumps({"prompt": user_query})
        )
        
        # Handle streaming response
        if "text/event-stream" in response.get("contentType", ""):
            content = []
            for line in response["response"].iter_lines(chunk_size=1):
                if line:
                    line = line.decode("utf-8")
                    if line.startswith("data: "):
                        content.append(line[6:])
            return "\n".join(content)
        else:
            # Handle non-streaming response
            events = []
            for event in response.get("response", []):
                events.append(event.decode("utf-8"))
            return json.loads(events[0]) if events else "No response received"
            
    except Exception as e:
        return f"❌ Error querying AgentCore Runtime: {str(e)}"

# Create Gradio interface
with gr.Blocks(title="PTO Agent - AgentCore Runtime", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🏖️ PTO Agent (AgentCore Runtime)")
    gr.Markdown("### Powered by Claude 3.5 Sonnet V2 on Amazon Bedrock AgentCore Runtime")
    
    gr.Markdown("""
    **Current PTO Balance: 94 hours (11.75 days)**
    
    This interface connects to your deployed AgentCore Runtime agent using boto3.
    Ask me anything about your PTO!
    """)
    
    with gr.Row():
        with gr.Column():
            user_input = gr.Textbox(
                label="💬 Ask about your PTO", 
                lines=3,
                placeholder="e.g., How many PTO hours do I have?"
            )
            submit_btn = gr.Button("🚀 Query AgentCore Runtime", variant="primary", size="lg")
        
    with gr.Row():
        agent_output = gr.Textbox(
            label="🤖 AgentCore Runtime Response", 
            lines=10,
            show_copy_button=True
        )
    
    # Example questions
    gr.Examples(
        examples=[
            "How many PTO hours do I have available?",
            "How many days of PTO can I take?",
            "Can I take a full day off next week?",
            "Do I have enough PTO for a half day?",
            "What's my PTO balance in days?",
            "Can I take 2 weeks off?"
        ],
        inputs=user_input,
        label="📝 Example Questions (click to try)"
    )
    
    # Information section
    with gr.Accordion("ℹ️ How This Works", open=False):
        gr.Markdown(f"""
        **Technology Stack:**
        - 🧠 **AI Model:** Claude 3.5 Sonnet V2 (Amazon Bedrock)
        - ⚙️ **Framework:** AWS Strands SDK
        - 🚀 **Runtime:** Amazon Bedrock AgentCore Runtime
        - 🔗 **Connection:** boto3 client
        - 🎨 **Interface:** Gradio
        
        **AgentCore Runtime Details:**
        - **Agent ARN:** {agent_arn or 'Not connected'}
        - **Region:** {region}
        - **Status:** {'Connected' if agent_arn else 'Disconnected'}
        
        **Available Tools:**
        1. `get_available_pto_hours()` - Returns total PTO hours
        2. `get_available_pto_days()` - Converts hours to days
        3. `can_take_full_day()` - Checks if ≥8 hours available
        4. `can_take_half_day()` - Checks if ≥4 hours available
        
        The agent runs in a containerized environment and is invoked via AWS API calls.
        """)
    
    submit_btn.click(
        fn=query_agentcore_runtime,
        inputs=user_input,
        outputs=agent_output
    )
    
    user_input.submit(
        fn=query_agentcore_runtime,
        inputs=user_input,
        outputs=agent_output
    )

if __name__ == "__main__":
    demo.launch(share=True)

In [ ]:
demo.close()

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# Congratulations!